In [131]:
import numpy as np
import pandas as pd
from gsheet import Data
import plotly.express as px

In [132]:
data = Data()
df = data.getData()
df

,Date,Expense Type,Budget,Expense Amount,Total Expense,Total Added Balance,Total Balance
0,2024-06-25,Food,2000,30,30,2040,2010
1,2024-06-25,Travel,1000,105,135,,1905


In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 2 non-null      object
 1   Expense Type         2 non-null      object
 2   Budget               2 non-null      int64 
 3   Expense Amount       2 non-null      int64 
 4   Total Expense        2 non-null      int64 
 5   Total Added Balance  2 non-null      object
 6   Total Balance        2 non-null      int64 
dtypes: int64(4), object(3)
memory usage: 244.0+ bytes


In [134]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Total Added Balance'] = df['Total Added Balance'].apply(lambda x: int(x) if x else 0)

In [135]:
df = df.sort_values('Date')

In [136]:
datewise = df.groupby('Date')['Expense Amount'].sum().reset_index()

In [137]:
datewise

,Date,Expense Amount
0,2024-06-25,135


In [138]:
px.line(data_frame=datewise, x='Date', y='Expense Amount', markers='o').update_layout(xaxis_title='Date', yaxis_title='Amount')

In [141]:
temp = df.copy()
temp['Month'] = temp['Date'].dt.month_name()
temp['Year'] = temp['Date'].dt.year
categorywise = temp.groupby(['Expense Type', 'Budget', 'Date', 'Month', 'Year'])['Expense Amount'].sum().reset_index()
categorywise

,Expense Type,Budget,Date,Month,Year,Expense Amount
0,Food,2000,2024-06-25,June,2024,30
1,Travel,1000,2024-06-25,June,2024,105


In [158]:
month = 'June'
year = 2024
categorywise[(categorywise['Month']=='June') & (categorywise['Year']==2024)]

,Expense Type,Budget,Date,Month,Year,Expense Amount
0,Food,2000,2024-06-25,June,2024,30
1,Travel,1000,2024-06-25,June,2024,105


In [159]:
categorywise.query(f'Month==\'{month}\' and Year=={year}')


,Expense Type,Budget,Date,Month,Year,Expense Amount
0,Food,2000,2024-06-25,June,2024,30
1,Travel,1000,2024-06-25,June,2024,105


In [142]:

df_melted = categorywise.melt(id_vars=["Expense Type", "Date"], value_vars=["Budget", "Expense Amount"], var_name="Category", value_name="Amount")

display(df_melted)

fig = px.bar(df_melted, x="Expense Type", y="Amount", color="Category", barmode="group").update_layout(yaxis_title='Budget/Expense Amount')


,Expense Type,Date,Category,Amount
0,Food,2024-06-25,Budget,2000
1,Travel,2024-06-25,Budget,1000
2,Food,2024-06-25,Expense Amount,30
3,Travel,2024-06-25,Expense Amount,105
